### 컨텐츠 기반 필터링(Content Based Filtering)

In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')
df1.columns=['id', 'title', 'cast', 'crew']
df = df2.merge(df1[['id', 'cast', 'crew']], on ='id') # id를 기준으로 병합
C = df2['vote_average'].mean()
m = df2['vote_count'].quantile(0.9) #상위 10% 해당하는 데이터 뽑음
q_movies = df.copy().loc[df2['vote_count'] >= m]
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / ( v + m ) * R ) + (m / ( m + v ) * C)
q_movies['score'] = q_movies.apply(weighted_rating, axis=1) # axis = 1 : row단위로 추가
q_movies = q_movies.sort_values('score', ascending=False)

In [2]:
df['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

overview - 줄거리<br>
줄거리 분석해서 비슷한 영화를 찾기

#### Bag of Words - BOW

문장 1 : I am a boy<br>
문장 2 : I am a girl<br>
I(2),am(2),a(2),boy(1),girl(1)<br>

        I       am      a       boy     girl 
문장1   1       1       1       1       0   (1,1,1,1,0)<br>
문장2   1       1       1       0       1   (1,1,1,0,1)<br>

피처 벡터화<br>
1. TfidfVectorizer (TF-IDF 기반의 벡터화)
- 각 문장에서 자주 나오는 단어들이 모든 문서에서 동일하게 많이 나온다면 패널티 적용
2. CountVectorizer(위와 같이 하나하나 다 센다.)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
# stop_words를 english로 하게되면 영어에서 필요없는 단어들을 불용어 처리

In [4]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [5]:
df['overview'].isnull().values.any()
# overview의 컬럼값에서 널에 해당하는(.isnull()) 데이터를 가져와서(.values) 하나라도 있으면 True 반영(.any())

True

In [6]:
df['overview']=df['overview'].fillna('') #fillna -> null 값을 ()안의 값으로 채우기

In [8]:
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape # 4803개의 문서들에 20978단어로 되어 있음

(4803, 20978)

In [9]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#cosine_similarity함수도 존재 위의 함수 쓰는 이유는 속도측면에서 위가 빠름
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [10]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates() # 중복제거
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [12]:
indices['The Dark Knight Rises']

3

In [13]:
df.iloc[[3]]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."


### 영화의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 영화 목록 반환

In [16]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값 얻기
    idx = indices[title]
    # 코사인 유사도 매트릭스에서 idx에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
    sim = list(enumerate(cosine_sim[idx]))
    # 람다식 예제 아래에
    sorted(sim, key = lambda x: x[1], reverse=True)
    # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
    sim = sim[1:11]
    # 추천 영화 목록 10개의 인덱스 
    movie_indice = [i[0] for i in sim]
    recommend_list = [df['title'].iloc[x] for x in movie_indice]
    return recommend_list


#### 람다식


In [14]:
def get_second(x):
    return x[1]
test_list = ['인덱스', '유사도']
print(get_second(test_list))

유사도


In [15]:
(lambda x: x[1])(test_list)

'유사도'

In [17]:
get_recommendations('The Avengers')

["Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns']